# NumPy with `rustimport`

For Google colab, we install [rustimport](https://github.com/thomasjpfan/rustimport) and the rust toolchain:

In [ ]:
import os
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install rustimport
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] += ":/root/.cargo/bin"

In [ ]:
%load_ext rustimport

## Simple NumPy Function Written in Rust

Based on the example from [PyO3/rust-numpy](https://github.com/PyO3/rust-numpy#example), we define a NumPy function that computes `a*x+y`:

In [ ]:
%%rustimport
//: [dependencies]
//: pyo3 = { version = "0.20", features = ["extension-module"] }
//: numpy = "0.20"

use pyo3::prelude::*;
use numpy::ndarray::{ArrayD, ArrayViewD};
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};

fn axpy(a: f64, x: ArrayViewD<'_, f64>, y: ArrayViewD<'_, f64>) -> ArrayD<f64> {
    a * &x + &y
}

#[pyfunction]
#[pyo3(name = "axpy")]
fn axpy_py<'py>(
    py: Python<'py>,
    a: f64,
    x: PyReadonlyArrayDyn<'py, f64>,
    y: PyReadonlyArrayDyn<'py, f64>,
) -> &'py PyArrayDyn<f64> {
    let x = x.as_array();
    let y = y.as_array();
    let z = axpy(a, x, y);
    z.into_pyarray(py)
}

Using `pyo3(name=axpy)`, the public python function is named `axpy`, which is available in the notebook environment:

In [ ]:
import numpy as np

a = 4.4
x = np.array([1.0, 3.0, 4.0], dtype=np.float64)
y = np.array([2.1, -1.0, -4.0], dtype=np.float64)

axpy(a, x, y)